In [2]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import statsmodels.tsa.api as smt
import statsmodels.api as sm

## 画时序图，ACF,PACF

In [3]:
def drawts(y,pname):
    ##draw ax
    fig = plt.figure(figsize=(10,8))
    ts_ax=plt.subplot2grid((2,2),(0,0),colspan=2)
    acf_ax=plt.subplot2grid((2,2),(1,0))
    pacf_ax=plt.subplot2grid((2,2),(1,1))
    ##draw plot
    ts_ax.plot(y,'*-')
    ts_ax.set_title('Time Series Analysis Plots')
    smt.graphics.plot_acf(y,lags=None,ax=acf_ax,alpha=0.05) ##2sigma
    smt.graphics.plot_pacf(y,lags=None,ax=pacf_ax,alpha=0.05)  ##2sigma
    #plt.savefig('%s.jpg'%pname,dpi=256)
    plt.show()
    plt.close()

## 计算季节指数

In [4]:
def Seasonal_index(y):
    S = np.zeros((12))
    for i in range(12):
        S[i] = np.mean(y[i::12]) / np.mean(y)
    print('Si:')
    print(S)
    plt.plot(np.arange(1, 12 + 1), S)
    plt.xlabel('month')
    plt.ylabel('Si')
    plt.title('Si')
    plt.show()
    return S

## 消除季节指数的影响

In [ ]:
def Eliminate(y,S):
    period=len(S)
    output=[0]*len(y)
    for i in range(0,len(y)):
        output[i]=y[i]/S[i%period]
    return output

## 增加季节指数的影响

In [ ]:
def Augment(y, S):
    period = len(S)
    output = [0]*len(y)
    for i in range(0,len(y)):
        output[i] = y[i]*S[i%period]
    return output

## 绘制对比图

In [ ]:
def plotcon(y,ypred,pname):
    plt.plot(y,'*-',label='observe')
    plt.plot(ypred,label='fitted')
    plt.legend()
    plt.title('observe and fitted')
    plt.show()

## 分析结果

In [ ]:
dfname='./时间序列分析/HW4-5/data1.11.csv'
y=pd.read_csv('%s'%dfname,header=None)
y.iloc[:, 0] = y.iloc[:, 0].astype('float')
y = y.values[:, 0]
drawts(y,dfname)
S=Seasonal_index(y)
output=Eliminate(y,S)
length=len(output)
y_axis=np.arange(length)
plt.title('Time Series Analysis Plots')
plt.plot(y_axis,output,'*-')
plt.show()
t = np.arange(1, len(output) + 1)
est = sm.OLS(y, sm.add_constant(t)).fit()
print(est.summary())
print('sigma2=%f' % est.resid.std() ** 2)
ypred = est.predict(sm.add_constant(t))
ypred=Augment(ypred,S)
plotcon(y, ypred, dfname)